#  ZeroShot Topic Modeling for Indic Languages (IndicCTM)
*Previously Zero-shot Cross-Lingual Topic Modeling For Same Script Languages*
> Do conxtextualized TM tackle zero-shot cross-lingual topic modeling better on same script languages?

We use 4000 documents as training and consider randomly sampled 800 documents as the test set. We collect the 800 respective instances in Hindi (hi), Assamese (as), Gujarati (gu), Kannada (kn), Malayalam (ml), Marathi (mr), Oriya (or), Punjabi (pa), Tamil (ta), Telugu (te) and English (en).

First, we use IndicBERT to generate multilingual embeddings as the input of the model. Then we evaluate multilingual topic predictions on the multilingual abstracts in test set.

## Hardware Check

1. Setup new env with py38 (anaconda)
2. Setup PyTorch + CUDA + GPU
https://medium.com/@leennewlife/how-to-setup-pytorch-with-cuda-in-windows-11-635dfa56724b

3. Setup Colab (Run in cwd\drustagi)
https://krishnacheedella.medium.com/connect-to-a-local-runtime-from-google-colab-to-your-local-jupyter-notebook-on-http-over-websocket-f7b4bd4eb585

4. Run Hardware Check

In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
4# CUDA Check (Local Runtime)
import torch
print(f'PyTorch version: {torch.__version__}')
print('*'*10)
print(f'_CUDA version: ')
!nvcc --version
print('*'*10)
print(f'CUDNN version: {torch.backends.cudnn.version()}')
print(f'Available GPU devices: {torch.cuda.device_count()}')
print(f'Device Name: {torch.cuda.get_device_name()}')

PyTorch version: 2.1.0a0+fe05266
**********
_CUDA version: 
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Feb__7_19:32:13_PST_2023
Cuda compilation tools, release 12.1, V12.1.66
Build cuda_12.1.r12.1/compiler.32415258_0
**********
CUDNN version: 8900
Available GPU devices: 1
Device Name: NVIDIA GeForce GTX 1660


## Setup

In [3]:
# Always use the most updated version of pip and jupyter notebook
!pip install --upgrade pip
!pip install --upgrade jupyter
!pip install --upgrade pyzmq
!pip install tensorflow==2.12.*

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


### Install Libraries and Imports

**Use Python Version 3.8.16**
> *3.10 does not support collections, so cannot use.*
>>Affected packages - inltk

In [4]:
# Install the contextualized topic model library
!pip install -U contextualized_topic_models

!pip install pyldavis
!pip install wget
!pip install head
!nvidia-smi

# Setup Hindi for analysis
!pip install indic-nlp-library==0.81
!pip install stopwordsiso
!pip install inltk
!pip install regex
!pip install urduhack

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 18.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 98.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.4/461.4 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 kB 105.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 784.3/784.3 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 30.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 39.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
# Imports
from contextualized_topic_models.models.ctm import ZeroShotTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import  WhiteSpacePreprocessingStopwords
import pickle

In [6]:
# Python3.10 issue doesn't work
# Either downgrade or use from collections.abc import Iterable
from inltk.inltk import setup
try:
  setup('hi')
except:
  pass

2023-05-14 22:45:14.966603: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 22:45:16.248157: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-14 22:45:16.248799: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## Load Data

**PMIndia Corpus**: 
Parallel corpus from the website of the Indian Prime Minister (www.pmindia.gov.in). 

We combine each speech document into one, for every language. Datasets are downloaded from [Statistical Machine Translation](https://data.statmt.org/pmindia/v1/monolingual/).

In [7]:
with open('train_speeches.pkl', 'rb') as train_s:
    train_speeches = pickle.load(train_s)

*Split Parallel Corpus Into Test and Train*

In [8]:
# Imports
import pandas as pd
from pprint import pprint
import re

# Selecting Train speeches
hindi_unprep = pd.DataFrame(list for list in train_speeches['hi'])
english_unprep = pd.DataFrame(list for list in train_speeches['en'])

# View each row as a speech
print(hindi_unprep[:5])
print(english_unprep[:5])

# n = 4003
print("\nHindi count:" + str(hindi_unprep.count()) + "\n")
# n = 4489
print("English count:" + str(english_unprep.count()) + "\n")

                                                   0
0  नागरिकों से स्वच्छाग्रही बनने और स्वच्छ भारत ब...
1  श्री सोमनाथ न्यास के न्यासियों की 116वीं बैठक ...
2  प्रधानमंत्री श्री नरेन्द्र मोदी की अध्यक्षता म...
3  प्रधानमंत्री श्री नरेन्द्र मोदी ने आज ही के दि...
4  · खूंटी की जिला अदालत में छत पर लगने वाले सौर ...
                                                   0
0  PM Calls upon citizens to become Swachhagrahis...
1  The first 10 months of Prime Minister Narendra...
2  BRICS in Africa: Collaboration for Inclusive G...
3  The 116th meeting of the trustees of Shri Somn...
4  Deendayal Upadhyaya Gram Jyoti Yojana\n The Un...

Hindi count:0    4003
dtype: int64

English count:0    4489
dtype: int64



# Training Models

## **Hindi-Based ZeroshotTM**

*Preprocessing*

Why do we use the preprocessed text here? We need text without punctuation to build the bag of word. Also, we might want only to have the most frequent words inside the BoW. Too many words might not help.

In [9]:
# Imports
from contextualized_topic_models.models.ctm import ZeroShotTM
import contextualized_topic_models.utils.data_preparation
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
import nltk
import pickle

**Modify WhiteSpacePreprocessingStopwords to support Hindi text**

Above class method is taken from author's original code and modified to support preprocessing of Hindi documents. Additional helper methods have been added to support the updated preprocessing() method.

FIXED: Returns expected preprocessed hindi texts.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
import string
from nltk.corpus import stopwords as stop_words
from gensim.utils import deaccent
import warnings
from inltk.inltk import remove_foreign_languages
import regex 
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize.indic_tokenize import trivial_tokenize

class WhiteSpacePreprocessingStopwords(WhiteSpacePreprocessingStopwords):
    """
    Overriden author's original code to keep accents during preprocessing.
    Provides a very simple preprocessing script that filters infrequent tokens from text
    """
    
    def tokenize_indic(self, sent, LANG = 'hi'):      
      normalizer_factory = IndicNormalizerFactory()
      normalizer = normalizer_factory.get_normalizer(LANG)
      normalized = normalizer.normalize(sent)
      tokenized = ' '.join(trivial_tokenize(normalized, LANG))
      return tokenized
    
    def custom_analyzer(self, text):
        """
        code source: https://stackoverflow.com/questions/60763030
                      having-an-issue-in-doing-count-vectorization-for-hindi-text
        """
        words = regex.findall(r'\w{2,}', text) # extract words of at least 2 letters
        for w in words:
            yield w

    def preprocess(self):
        """
        Note that if after filtering some documents do not contain words we remove them. That is why we return also the
        list of unpreprocessed documents.
        :return: preprocessed documents, unpreprocessed documents and the vocabulary list
        """
        preprocessed_docs_tmp = self.documents
        preprocessed_docs_tmp = [doc.translate(
            str.maketrans(string.punctuation, ' ' * len(string.punctuation))) for doc in preprocessed_docs_tmp]
        if self.remove_numbers:
            preprocessed_docs_tmp = [doc.translate(str.maketrans("0123456789", ' ' * len("0123456789")))
                                     for doc in preprocessed_docs_tmp]
        preprocessed_docs_tmp = [' '.join([w for w in doc.split() if len(w) > 0 and w not in self.stopwords])
                                 for doc in preprocessed_docs_tmp]
     
        preprocessed_docs_tmp = [self.tokenize_indic(sent) for sent in preprocessed_docs_tmp]
        
        # USE CUSTOM ANALYZER TO PRESERVE ACCENTS IN VECTORIZER
        vectorizer = CountVectorizer(analyzer = self.custom_analyzer, max_features=self.vocabulary_size, max_df=self.max_df)
        vectorizer.fit_transform(preprocessed_docs_tmp)
        '''
        # 04/24 REPLACE ATTRIBUTE
            The get_feature_names attribute was deprecated in scikit-learn 1.0 and removed in scikit-learn 1.1. 
            To get the feature names, you can use the get_feature_names_out attribute instead.
        '''
        temp_vocabulary = set(vectorizer.get_feature_names_out()) 

        preprocessed_docs_tmp = [' '.join([w for w in doc.split() if w in temp_vocabulary])
                                 for doc in preprocessed_docs_tmp]

        preprocessed_docs, unpreprocessed_docs, retained_indices = [], [], []
        for i, doc in enumerate(preprocessed_docs_tmp):
            if len(doc) > 0 and len(doc) >= self.min_words:
                preprocessed_docs.append(doc)
                unpreprocessed_docs.append(self.documents[i])
                retained_indices.append(i)

        vocabulary = list(set([item for doc in preprocessed_docs for item in doc.split()]))

        return preprocessed_docs, unpreprocessed_docs, vocabulary, retained_indices                                              

We'll only do this for the first time.


### Prepare Data for Modeling

We don't discard the non-preprocessed hindi texts, because we are going to use them as input for obtaining the **contextualized** document representations.

In [11]:
### HINDI ###
LANG_SELECTED = 'hi'

# We select 200 tokens per speech
NUM_TOKENS = 200

# Import Hindi Stopwords
import stopwordsiso as stopwords

# Run preprocessing script
documents = [line[:NUM_TOKENS].strip() for line in train_speeches[LANG_SELECTED]]
sp = WhiteSpacePreprocessingStopwords(documents, stopwords_list = stopwords.stopwords(LANG_SELECTED))
preprocessed_documents, unpreprocessed_corpus, vocab, __ = sp.preprocess()

# Ensure same length for preprocessed and unpreprocessed
print(len(preprocessed_documents), len(unpreprocessed_corpus))

4003 4003


In [12]:
from pprint import pprint
pprint(preprocessed_documents[:5])

['नागरिकों बनने स्वच्छ भारत बनाने आह्वान प्रधानमंत्री श्री नरेन्द्र मोदी '
 'महात्मा गांधी साल पूरे अवसर राष्ट्रीय राजधानी',
 'श्री वीं बैठक आज सम्पन्न हुई बैठक प्रधानमंत्री श्री नरेंद्र मोदी श्री आडवाणी '
 'श्री शाह श्री भाई पटेल श्री पी',
 'प्रधानमंत्री श्री नरेन्द्र मोदी अध्यक्षता मंत्रीमंडल बैठक दीनदयाल उपाध्याय '
 'ग्राम योजना प्रारंभ आज अनुमति दी गई योजना तहत ग्रामीण क्षेत्रों',
 'प्रधानमंत्री श्री नरेन्द्र मोदी आज दिन मुंबई आतंकी हमलों दौरान शहीद सलाम '
 'जिन्होंने जिंदगी दे',
 'जिला सौर ऊर्जा संयंत्र उद्घाटन किया प्रधानमंत्री मुद्रा योजना विशाल ऋण '
 'उद्घाटन लाभार्थियों ऋण दस्तावेज मुद्रा सौ']


**TopicModelDataPreparation**

We will now pass our files with preprocess and unpreprocessed data to our TopicModelDataPreparation object. This object takes care of creating the bag of words and obtains the contextualized BERT representations of documents. This operation allows us to create our training dataset.

**Modify TopicModelDataPrepation to preserve accents in vectorizer.**

In [13]:
from contextualized_topic_models.utils import data_preparation as dp
import numpy as np
from sentence_transformers import SentenceTransformer
import scipy.sparse
import warnings
from contextualized_topic_models.datasets.dataset import CTMDataset
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder

class TopicModelDataPreparation(TopicModelDataPreparation):
      def custom_analyzer(self, text):
        """
        code source: https://stackoverflow.com/questions/60763030
                  having-an-issue-in-doing-count-vectorization-for-hindi-text
        """
        words = regex.findall(r'\w{2,}', text) # extract words of at least 2 letters
        for w in words:
          yield w

      def fit(self, text_for_contextual, text_for_bow, labels=None, custom_embeddings=None):
        """
        This method fits the vectorizer and gets the embeddings from the contextual model
        :param text_for_contextual: list of unpreprocessed documents to generate the contextualized embeddings
        :param text_for_bow: list of preprocessed documents for creating the bag-of-words
        :param custom_embeddings: np.ndarray type object to use custom embeddings (optional).
        :param labels: list of labels associated with each document (optional).
        """

        if custom_embeddings is not None:
            assert len(text_for_contextual) == len(custom_embeddings)

            if text_for_bow is not None:
                assert len(custom_embeddings) == len(text_for_bow)

            if type(custom_embeddings).__module__ != 'numpy':
                raise TypeError("contextualized_embeddings must be a numpy.ndarray type object")

        if text_for_bow is not None:
            assert len(text_for_contextual) == len(text_for_bow)

        if self.contextualized_model is None and custom_embeddings is None:
            raise Exception("A contextualized model or contextualized embeddings must be defined")

        # TODO: this count vectorizer removes tokens that have len = 1, might be unexpected for the users
        self.vectorizer = CountVectorizer(analyzer = self.custom_analyzer)

        train_bow_embeddings = self.vectorizer.fit_transform(text_for_bow)

        # if the user is passing custom embeddings we don't need to create the embeddings using the model
        if custom_embeddings is None:
            train_contextualized_embeddings = dp.bert_embeddings_from_list(
                text_for_contextual, sbert_model_to_load=self.contextualized_model, max_seq_length=self.max_seq_length)
        else:
            train_contextualized_embeddings = custom_embeddings
        self.vocab = self.vectorizer.get_feature_names_out()
        self.id2token = {k: v for k, v in zip(range(0, len(self.vocab)), self.vocab)}

        if labels:
            self.label_encoder = OneHotEncoder()
            encoded_labels = self.label_encoder.fit_transform(np.array([labels]).reshape(-1, 1))
        else:
            encoded_labels = None
        return CTMDataset(
            X_contextual=train_contextualized_embeddings, X_bow=train_bow_embeddings,
            idx2token=self.id2token, labels=encoded_labels)

In [14]:
# Directory to store models
!mkdir models

mkdir: cannot create directory ‘models’: File exists


In [15]:
MODELS_PATH = os.getcwd() + "/models"
MODELS_PATH

'/workspace/models'

*Here we use the contextualized model "ai4bharat/indic-bert", because we need a **multilingual model for indic languages** for performing cross-lingual predictions later.*

In [16]:
# Load Indic Multilingual embeddings 
tp = TopicModelDataPreparation('ai4bharat/indic-bert')
tp.max_seq_length = 200

In [17]:
# Building training dataset
training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)

No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/ai4bharat_indic-bert. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/ai4bharat_indic-bert were not used when initializing AlbertModel: ['predictions.dense.bias', 'predictions.decoder.weight', 'sop_classifier.classifier.weight', 'predictions.LayerNorm.bias', 'sop_classifier.classifier.bias', 'predictions.decoder.bias', 'predictions.dense.weight', 'predictions.bias', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassi

Batches:   0%|          | 0/21 [00:00<?, ?it/s]

### Training 25 and 50 topics models

In [18]:
# Train over 100 epochs
'''
    05/14/23: .save() causing issues, using joblib to save models instead
    '''
# Save model 
import joblib 

### HINDI : 25 TOPICS ###
z_ctm_25_HI = ZeroShotTM(bow_size=len(tp.vocab), n_components = 25, contextual_size=768, num_epochs=100)

z_ctm_25_HI.fit(training_dataset, n_samples = 30) # run the model
#z_ctm_25_HI.save("./") # save the model
joblib.dump(z_ctm_25_HI, MODELS_PATH + '/z_ctm_25_HI.pkl')

# ### HINDI : 50 TOPICS ###
z_ctm_50_HI = ZeroShotTM(bow_size=len(tp.vocab), n_components = 50, contextual_size=768, num_epochs=100)

z_ctm_50_HI.fit(training_dataset, n_samples = 30) # run the model
#z_ctm_50_HI.save("./") # save the model
joblib.dump(z_ctm_50_HI, MODELS_PATH + '/z_ctm_50_HI.pkl')

Epoch: [100/100]	 Seen Samples: [396800/400300]	Train Loss: 140.4304196757655	Time: 0:00:00.574728: : 100it [00:54,  1.82it/s]
100%|██████████████████████████████████████████████████████████████████████████| 63/63 [00:00<00:00, 157.87it/s]
Epoch: [100/100]	 Seen Samples: [396800/400300]	Train Loss: 148.09137135167276	Time: 0:00:00.523492: : 100it [00:55,  1.80it/s]
100%|██████████████████████████████████████████████████████████████████████████| 63/63 [00:00<00:00, 155.79it/s]


['/workspace/models/z_ctm_50_HI.pkl']

### Topic Predictions

**PROBLEM - RESOLVED**
Predicted topics do not make sense in the train language - Hindi. Preprocessing works just fine. Something is getting lost in translation during model fit. 

* PATH 1: Topics are inferences from text. (NLI?)
* PATH 2: Topics are just words from within the text. (Extraction)
* PATH 3: Topics are lemmetized versions of preprocessed tokens. 


> *What is topic modeling?*

Topic modeling is an unsupervised machine learning technique that’s capable of scanning a set of documents, detecting word and phrase patterns within them, and automatically clustering word groups and similar expressions that best characterize a set of documents.

src: Google

In [19]:
# See topic predictions per speech doc
z_ctm_25_HI.get_topic_lists(5)[:4]

[['मंजूरी', 'दी', 'दे', 'अध्यक्षता', 'केन्द्रीय'],
 ['हैं', 'संदेश', 'शुभकामनाएं', 'लोगों', 'अवसर'],
 ['की', 'मुलाकात', 'आज', 'श्री', 'बातचीत'],
 ['बहनों', 'संख्या', 'विशाल', 'प्यारे', 'भाइयों']]

In [20]:
# See topic predictions per speech doc
z_ctm_50_HI.get_topic_lists(5)[:4]

[['Excellency', 'President', 'of', 'the', 'Your'],
 ['की', 'मुलाकात', 'आज', 'श्री', 'बातचीत'],
 ['श्री', 'प्रधानमंत्री', 'मोदी', 'निधन', 'बधाई'],
 ['बजट', 'देश', 'सत्र', 'संसद', 'होगा']]

In [21]:
# Save Output in .csv files
'''
The string contains non-ASCII characters, since text is in Hindi.
Open file in a mode that supports Unicode by specifying the encoding when you open the file. 
'''
hi_topics_25 = open("hi_topics_25.csv","w", encoding='utf-8')
hi_topics_50 = open("hi_topics_50.csv","w", encoding='utf-8')

import csv
wr1 = csv.writer(hi_topics_25)
wr1.writerows(z_ctm_25_HI.get_topic_lists(5))

wr2 = csv.writer(hi_topics_50)
wr2.writerows(z_ctm_50_HI.get_topic_lists(5))

### NPMI Coherence for Hindi-Based ZeroShotTM

In [23]:
# Get NPMI Coherence
from contextualized_topic_models.evaluation.measures import CoherenceNPMI
texts = [doc.split() for doc in preprocessed_documents] # load text for NPMI

### 25 TOPICS ###
npmi_HI = CoherenceNPMI(texts=texts, topics=z_ctm_25_HI.get_topic_lists(25))
#print(npmi_HI.score())

### 50 TOPICS ###
npmi_50_HI = CoherenceNPMI(texts=texts, topics=z_ctm_50_HI.get_topic_lists(50))
#print(npmi_50_HI.score())

# Store NPMI scores
zeroshotNPMI_HI = [npmi_HI.score(), npmi_50_HI.score()]

# SHOW RESULTS
NPMI_HI = {"ZeroShotTM for Hindi" : zeroshotNPMI_HI,}

# Set new axis labels and print the DataFrame
npmi_HI = pd.DataFrame.from_dict(NPMI_HI, orient='index')

print("NPMI Coherences")
npmi_HI.set_axis(["t(25)", "t(50)"], axis = 1)

NPMI Coherences


,t(25),t(50)
ZeroShotTM for Hindi,0.168529,0.171727


## **English-Based ZeroshotTM**

### Prepare Data for Modeling

In [24]:
### ENGLISH ###
LANG_SELECTED = 'en'

# We select 200 tokens per speech
NUM_TOKENS = 200

# Download English Stopwords
nltk.download('stopwords')

# Run preprocessing script
documents = [line[:NUM_TOKENS].strip() for line in train_speeches[LANG_SELECTED]]

import nltk
nltk.download('words')
# preprocessed_documents
words = set(nltk.corpus.words.words())

documents = ["".join([word for word in doc if word.lower() in words or not word.isalpha()]) for doc in documents]

sp = WhiteSpacePreprocessing(documents, stopwords_language='english')
preprocessed_documents, unpreprocessed_corpus, vocab, __ = sp.preprocess()

# Ensure same length for preprocessed and unpreprocessed
print(len(preprocessed_documents), len(unpreprocessed_corpus))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


4107 4107


In [25]:
from pprint import pprint
pprint(preprocessed_documents[:5])

['pm calls upon citizens become create swachh bharat prime minister shri '
 'narendra modi inaugurate exhibition titled ek abhiyan',
 'first 10 months prime minister narendra term marked high international '
 'visits india us president barack obama xi',
 'brics africa growth shared prosperity 4th industrial convention centre south '
 '27 july 2018 10th brics summit',
 'meeting shri trust held today meeting attended prime minister shri narendra '
 'modi shri lal krishna advani shri sh',
 'deendayal upadhyaya yojana union cabinet chaired prime minister shri '
 'narendra modi today approved launch deendayal upadhyaya yojana']


In [26]:
# Load English embeddings
tp = TopicModelDataPreparation("sentence-transformers/bert-base-nli-mean-tokens")
tp.max_seq_length = 200

In [27]:
# Building training dataset
en_training = tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)

/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_com

/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_com

/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_com

/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_com

/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_com

...

/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_com

/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_com

/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_com

/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(
/usr/local/lib/python3.8/dist-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_com

Batches:   0%|          | 0/21 [00:00<?, ?it/s]

### Training 25 and 50 topics models

In [28]:
# Train over 100 epochs

### ENGLISH : 25 TOPICS ###
z_ctm_25_EN = ZeroShotTM(bow_size=len(tp.vocab), n_components = 25, contextual_size=768, num_epochs=100)
z_ctm_25_EN.fit(en_training, n_samples=30) # run the model

#z_ctm_25_EN.save("./") # save the model
joblib.dump(z_ctm_25_EN, MODELS_PATH + '/z_ctm_25_EN.pkl')

### ENGLISH : 50 TOPICS ###
z_ctm_50_EN = ZeroShotTM(bow_size=len(tp.vocab), n_components = 50, contextual_size=768, num_epochs=100)
z_ctm_50_EN.fit(en_training, n_samples=30) # run the model

#z_ctm_50_EN.save("./") # save the model
joblib.dump(z_ctm_50_EN, MODELS_PATH + '/z_ctm_50_EN.pkl')

Epoch: [100/100]	 Seen Samples: [409600/410700]	Train Loss: 119.7231285572052	Time: 0:00:00.595500: : 100it [01:04,  1.55it/s]
100%|██████████████████████████████████████████████████████████████████████████| 65/65 [00:00<00:00, 128.21it/s]
Epoch: [100/100]	 Seen Samples: [409600/410700]	Train Loss: 127.82322633266449	Time: 0:00:00.641278: : 100it [01:05,  1.53it/s]
100%|██████████████████████████████████████████████████████████████████████████| 65/65 [00:00<00:00, 128.18it/s]


['/workspace/models/z_ctm_50_EN.pkl']

### Topic Predictions

In [29]:
# See topic predictions per speech doc
z_ctm_25_EN.get_topic_lists(5)[:4]

[['people', 'day', 'greeted', 'greetings', 'occasion'],
 ['approval', 'given', 'chaired', 'union', 'cabinet'],
 ['today', 'development', 'meeting', 'projects', 'visited'],
 ['chaired', 'union', 'cabinet', 'mou', 'memorandum']]

In [30]:
# See topic predictions per speech doc
z_ctm_50_EN.get_topic_lists(5)[:4]

[['cabinet', 'union', 'chaired', 'understanding', 'memorandum'],
 ['tomorrow', 'inaugurate', 'new', 'arrive', 'delhi'],
 ['minister', 'prime', 'nepal', 'shri', 'nawaz'],
 ['condoled', 'passing', 'demise', 'away', 'shri']]

In [31]:
# Save Output in .csv files
'''
The string contains non-ASCII characters, since text is in Hindi.
Open file in a mode that supports Unicode by specifying the encoding when you open the file. 
'''
en_topics_25 = open("en_topics_25.csv","w", encoding='utf-8')
en_topics_50 = open("en_topics_50.csv","w", encoding='utf-8')

import csv
wr1 = csv.writer(en_topics_25)
wr1.writerows(z_ctm_25_EN.get_topic_lists(5))

wr2 = csv.writer(en_topics_50)
wr2.writerows(z_ctm_50_EN.get_topic_lists(5))

### NPMI Coherence for English-Based ZeroShotTM

In [32]:
# Get NPMI Coherence
from contextualized_topic_models.evaluation.measures import CoherenceNPMI
texts = [doc.split() for doc in preprocessed_documents] # load text for NPMI

### 25 TOPICS ###-
npmi_EN = CoherenceNPMI(texts=texts, topics=z_ctm_25_EN.get_topic_lists(25))
# print(npmi_EN.score())

### 50 TOPICS ###
npmi_50_EN = CoherenceNPMI(texts=texts, topics=z_ctm_50_EN.get_topic_lists(50))
# print(npmi_50_EN.score())

# Store NPMI scores
zeroshotNPMI_EN = [npmi_EN.score(), npmi_50_EN.score()]

# SHOW RESULTS
NPMI_EN = {"ZeroShotTM for English" : zeroshotNPMI_EN}

npmi_EN = pd.DataFrame.from_dict(NPMI_EN, orient='index')
print("NPMI Coherences")
npmi_EN.set_axis(["t(25)", "t(50)"], axis = 1)

NPMI Coherences


,t(25),t(50)
ZeroShotTM for English,0.105569,0.146268


# Coherence Results

In [33]:
# SHOW RESULTS
NPMI = {"ZeroShotTM for Hindi" : zeroshotNPMI_HI,
        "ZeroShotTM for English" : zeroshotNPMI_EN}

npmi = pd.DataFrame.from_dict(NPMI, orient='index')
print("NPMI Coherences")
npmi.set_axis(["t(25)", "t(50)"], axis = 1)

NPMI Coherences


,t(25),t(50)
ZeroShotTM for Hindi,0.168529,0.171727
ZeroShotTM for English,0.105569,0.146268


# Predictions and Evaluation
## **Unseen Multilingual  Corpora Predictions**

*Languages*

* Assamese - as
* Bengali - bn
* English - en
* Gujarati - gu
* Hindi - hi
* Kannada - kn
* Malayalam - ml
* Marathi - mr
* Oriya - or
* Punjabi - pa
* Tamil - ta
* Telugu - te

## Prepare Testsets for Indic Languages

In [ ]:
# Load Data
with open('parallel_speeches.pkl', 'rb') as test_s:
    parallel_speeches = pickle.load(test_s)

In [34]:
# Convert test files into test datasets
as_testset = tp.transform(parallel_speeches['as'])
bn_testset = tp.transform(parallel_speeches['bn'])
en_testset = tp.transform(parallel_speeches['en'])
gu_testset = tp.transform(parallel_speeches['gu'])
hi_testset = tp.transform(parallel_speeches['hi'])
kn_testset = tp.transform(parallel_speeches['kn'])
ml_testset = tp.transform(parallel_speeches['ml'])
mr_testset = tp.transform(parallel_speeches['mr'])
or_testset = tp.transform(parallel_speeches['or'])
pa_testset = tp.transform(parallel_speeches['pa'])
ta_testset = tp.transform(parallel_speeches['ta'])
te_testset = tp.transform(parallel_speeches['te'])

NameError: name 'parallel_speeches' is not defined

### Hindi Topic Predictions

In [ ]:
### HINDI : 25 TOPIC PREDICTIONS ### 
as_topics_predictions = z_ctm_25_HI.get_thetas(as_testset, n_samples=100) # get all the topic predictions
bn_topics_predictions = z_ctm_25_HI.get_thetas(bn_testset, n_samples=100) # get all the topic predictions
en_topics_predictions = z_ctm_25_HI.get_thetas(en_testset, n_samples=100) # get all the topic predictions
gu_topics_predictions = z_ctm_25_HI.get_thetas(gu_testset, n_samples=100) # get all the topic predictions
hi_topics_predictions = z_ctm_25_HI.get_thetas(hi_testset, n_samples=100) # get all the topic predictions
kn_topics_predictions = z_ctm_25_HI.get_thetas(kn_testset, n_samples=100) # get all the topic predictions
ml_topics_predictions = z_ctm_25_HI.get_thetas(ml_testset, n_samples=100) # get all the topic predictions
mr_topics_predictions = z_ctm_25_HI.get_thetas(mr_testset, n_samples=100) # get all the topic predictions
or_topics_predictions = z_ctm_25_HI.get_thetas(or_testset, n_samples=100) # get all the topic predictions
pa_topics_predictions = z_ctm_25_HI.get_thetas(pa_testset, n_samples=100) # get all the topic predictions
ta_topics_predictions = z_ctm_25_HI.get_thetas(ta_testset, n_samples=100) # get all the topic predictions
te_topics_predictions = z_ctm_25_HI.get_thetas(te_testset, n_samples=100) # get all the topic predictions

topics_25_HI = {'as': as_topics_predictions, 'bn': bn_topics_predictions, 
             'en': en_topics_predictions, 'gu': gu_topics_predictions,
             'hi': hi_topics_predictions, 'kn': kn_topics_predictions,
             'ml': ml_topics_predictions, 'mr': mr_topics_predictions,
             'or': or_topics_predictions, 'pa': pa_topics_predictions,
             'ta': ta_topics_predictions, 'te': te_topics_predictions}

In [ ]:
### HINDI : 50 TOPIC PREDICTIONS ### 
as_topics_predictions = z_ctm_50_HI.get_thetas(as_testset, n_samples=100) # get all the topic predictions
bn_topics_predictions = z_ctm_50_HI.get_thetas(bn_testset, n_samples=100) # get all the topic predictions
en_topics_predictions = z_ctm_50_HI.get_thetas(en_testset, n_samples=100) # get all the topic predictions
gu_topics_predictions = z_ctm_50_HI.get_thetas(gu_testset, n_samples=100) # get all the topic predictions
hi_topics_predictions = z_ctm_50_HI.get_thetas(hi_testset, n_samples=100) # get all the topic predictions
kn_topics_predictions = z_ctm_50_HI.get_thetas(kn_testset, n_samples=100) # get all the topic predictions
ml_topics_predictions = z_ctm_50_HI.get_thetas(ml_testset, n_samples=100) # get all the topic predictions
mr_topics_predictions = z_ctm_50_HI.get_thetas(mr_testset, n_samples=100) # get all the topic predictions
or_topics_predictions = z_ctm_50_HI.get_thetas(or_testset, n_samples=100) # get all the topic predictions
pa_topics_predictions = z_ctm_50_HI.get_thetas(pa_testset, n_samples=100) # get all the topic predictions
ta_topics_predictions = z_ctm_50_HI.get_thetas(ta_testset, n_samples=100) # get all the topic predictions
te_topics_predictions = z_ctm_50_HI.get_thetas(te_testset, n_samples=100) # get all the topic predictions

topics_50_HI = {'as': as_topics_predictions, 'bn': bn_topics_predictions, 
             'en': en_topics_predictions, 'gu': gu_topics_predictions,
             'hi': hi_topics_predictions, 'kn': kn_topics_predictions,
             'ml': ml_topics_predictions, 'mr': mr_topics_predictions,
             'or': or_topics_predictions, 'pa': pa_topics_predictions,
             'ta': ta_topics_predictions, 'te': te_topics_predictions}

### English Topic Predictions

In [ ]:
### ENGLISH : 25 TOPIC PREDICTIONS ### 
as_topics_predictions = z_ctm_25_EN.get_thetas(as_testset, n_samples=100) # get all the topic predictions
bn_topics_predictions = z_ctm_25_EN.get_thetas(bn_testset, n_samples=100) # get all the topic predictions
en_topics_predictions = z_ctm_25_EN.get_thetas(en_testset, n_samples=100) # get all the topic predictions
gu_topics_predictions = z_ctm_25_EN.get_thetas(gu_testset, n_samples=100) # get all the topic predictions
hi_topics_predictions = z_ctm_25_EN.get_thetas(hi_testset, n_samples=100) # get all the topic predictions
kn_topics_predictions = z_ctm_25_EN.get_thetas(kn_testset, n_samples=100) # get all the topic predictions
ml_topics_predictions = z_ctm_25_EN.get_thetas(ml_testset, n_samples=100) # get all the topic predictions
mr_topics_predictions = z_ctm_25_EN.get_thetas(mr_testset, n_samples=100) # get all the topic predictions
or_topics_predictions = z_ctm_25_EN.get_thetas(or_testset, n_samples=100) # get all the topic predictions
pa_topics_predictions = z_ctm_25_EN.get_thetas(pa_testset, n_samples=100) # get all the topic predictions
ta_topics_predictions = z_ctm_25_EN.get_thetas(ta_testset, n_samples=100) # get all the topic predictions
te_topics_predictions = z_ctm_25_EN.get_thetas(te_testset, n_samples=100) # get all the topic predictions

topics_25_EN = {'as': as_topics_predictions, 'bn': bn_topics_predictions, 
             'en': en_topics_predictions, 'gu': gu_topics_predictions,
             'hi': hi_topics_predictions, 'kn': kn_topics_predictions,
             'ml': ml_topics_predictions, 'mr': mr_topics_predictions,
             'or': or_topics_predictions, 'pa': pa_topics_predictions,
             'ta': ta_topics_predictions, 'te': te_topics_predictions}

In [ ]:
### ENGLISH : 50 TOPIC PREDICTIONS ### 
as_topics_predictions = z_ctm_50_EN.get_thetas(as_testset, n_samples=100) # get all the topic predictions
bn_topics_predictions = z_ctm_50_EN.get_thetas(bn_testset, n_samples=100) # get all the topic predictions
en_topics_predictions = z_ctm_50_EN.get_thetas(en_testset, n_samples=100) # get all the topic predictions
gu_topics_predictions = z_ctm_50_EN.get_thetas(gu_testset, n_samples=100) # get all the topic predictions
hi_topics_predictions = z_ctm_50_EN.get_thetas(hi_testset, n_samples=100) # get all the topic predictions
kn_topics_predictions = z_ctm_50_EN.get_thetas(kn_testset, n_samples=100) # get all the topic predictions
ml_topics_predictions = z_ctm_50_EN.get_thetas(ml_testset, n_samples=100) # get all the topic predictions
mr_topics_predictions = z_ctm_50_EN.get_thetas(mr_testset, n_samples=100) # get all the topic predictions
or_topics_predictions = z_ctm_50_EN.get_thetas(or_testset, n_samples=100) # get all the topic predictions
pa_topics_predictions = z_ctm_50_EN.get_thetas(pa_testset, n_samples=100) # get all the topic predictions
ta_topics_predictions = z_ctm_50_EN.get_thetas(ta_testset, n_samples=100) # get all the topic predictions
te_topics_predictions = z_ctm_50_EN.get_thetas(te_testset, n_samples=100) # get all the topic predictions

topics_50_EN = {'as': as_topics_predictions, 'bn': bn_topics_predictions, 
             'en': en_topics_predictions, 'gu': gu_topics_predictions,
             'hi': hi_topics_predictions, 'kn': kn_topics_predictions,
             'ml': ml_topics_predictions, 'mr': mr_topics_predictions,
             'or': or_topics_predictions, 'pa': pa_topics_predictions,
             'ta': ta_topics_predictions, 'te': te_topics_predictions}

# **Quantitative Evaluation**

In [ ]:
# Import metrics
from contextualized_topic_models.evaluation.measures import Matches, KLDivergence, CentroidDistance
import warnings
warnings.filterwarnings('ignore')

1. **Matches**

> Matches is the % of times the predicted topic for the non-English test document is the same as for the respective test document in English. The higher the scores, the better.

## Matches

### Hindi

In [ ]:
# HINDI : Matches for 25 topics
hi_as_matches = Matches(topics_25_HI['hi'], topics_25_HI['as'])
hi_bn_matches = Matches(topics_25_HI['hi'], topics_25_HI['bn'])
hi_en_matches = Matches(topics_25_HI['hi'], topics_25_HI['en'])
hi_gu_matches = Matches(topics_25_HI['hi'], topics_25_HI['gu'])
hi_kn_matches = Matches(topics_25_HI['hi'], topics_25_HI['kn'])
hi_ml_matches = Matches(topics_25_HI['hi'], topics_25_HI['ml'])
hi_mr_matches = Matches(topics_25_HI['hi'], topics_25_HI['mr'])
hi_or_matches = Matches(topics_25_HI['hi'], topics_25_HI['or'])
hi_pa_matches = Matches(topics_25_HI['hi'], topics_25_HI['pa'])
hi_ta_matches = Matches(topics_25_HI['hi'], topics_25_HI['ta'])
hi_te_matches = Matches(topics_25_HI['hi'], topics_25_HI['te'])


matches_25_HI = {'as': hi_as_matches.score(), 'bn': hi_bn_matches.score(), 
             'en': hi_en_matches.score(), 'gu': hi_gu_matches.score(),
             'kn': hi_kn_matches.score(),
             'ml': hi_ml_matches.score(), 'mr': hi_mr_matches.score(),
             'or': hi_or_matches.score(), 'pa': hi_pa_matches.score(),
             'ta': hi_ta_matches.score(), 'te': hi_te_matches.score()}
matches_25_HI

In [ ]:
# Average Matches for 25 topics
average_matches_25_HI = sum(matches_25_HI.values())/len(matches_25_HI.keys())

In [ ]:
# HINDI : Matches for 50 topics
hi_as_matches = Matches(topics_50_HI['hi'], topics_50_HI['as'])
hi_bn_matches = Matches(topics_50_HI['hi'], topics_50_HI['bn'])
hi_en_matches = Matches(topics_50_HI['hi'], topics_50_HI['en'])
hi_gu_matches = Matches(topics_50_HI['hi'], topics_50_HI['gu'])
hi_kn_matches = Matches(topics_50_HI['hi'], topics_50_HI['kn'])
hi_ml_matches = Matches(topics_50_HI['hi'], topics_50_HI['ml'])
hi_mr_matches = Matches(topics_50_HI['hi'], topics_50_HI['mr'])
hi_or_matches = Matches(topics_50_HI['hi'], topics_50_HI['or'])
hi_pa_matches = Matches(topics_50_HI['hi'], topics_50_HI['pa'])
hi_ta_matches = Matches(topics_50_HI['hi'], topics_50_HI['ta'])
hi_te_matches = Matches(topics_50_HI['hi'], topics_50_HI['te'])


matches_50_HI = {'as': hi_as_matches.score(), 'bn': hi_bn_matches.score(), 
             'en': hi_en_matches.score(), 'gu': hi_gu_matches.score(),
             'kn': hi_kn_matches.score(),
             'ml': hi_ml_matches.score(), 'mr': hi_mr_matches.score(),
             'or': hi_or_matches.score(), 'pa': hi_pa_matches.score(),
             'ta': hi_ta_matches.score(), 'te': hi_te_matches.score()}
matches_50_HI

In [ ]:
# Average Matches for 50 topics
average_matches_50_HI = sum(matches_50_HI.values())/len(matches_50_HI.keys())

### English

In [ ]:
# ENGLISH : Matches for 25 topics
en_as_matches = Matches(topics_25_EN['en'], topics_25_EN['as'])
en_bn_matches = Matches(topics_25_EN['en'], topics_25_EN['bn'])
en_hi_matches = Matches(topics_25_EN['en'], topics_25_EN['hi'])
en_gu_matches = Matches(topics_25_EN['en'], topics_25_EN['gu'])
en_kn_matches = Matches(topics_25_EN['en'], topics_25_EN['kn'])
en_ml_matches = Matches(topics_25_EN['en'], topics_25_EN['ml'])
en_mr_matches = Matches(topics_25_EN['en'], topics_25_EN['mr'])
en_or_matches = Matches(topics_25_EN['en'], topics_25_EN['or'])
en_pa_matches = Matches(topics_25_EN['en'], topics_25_EN['pa'])
en_ta_matches = Matches(topics_25_EN['en'], topics_25_EN['ta'])
en_te_matches = Matches(topics_25_EN['en'], topics_25_EN['te'])


matches_25_EN = {'as': en_as_matches.score(), 'bn': en_bn_matches.score(), 
             'en': en_hi_matches.score(), 'gu': en_gu_matches.score(),
             'kn': en_kn_matches.score(),
             'ml': en_ml_matches.score(), 'mr': en_mr_matches.score(),
             'or': en_or_matches.score(), 'pa': en_pa_matches.score(),
             'ta': en_ta_matches.score(), 'te': en_te_matches.score()}
matches_25_EN

In [ ]:
# Average Matches for 25 topics
average_matches_25_EN = sum(matches_25_EN.values())/len(matches_25_EN.keys())

In [ ]:
# ENGLISH : Matches for 50 topics
en_as_matches = Matches(topics_50_EN['en'], topics_50_EN['as'])
en_bn_matches = Matches(topics_50_EN['en'], topics_50_EN['bn'])
en_en_matches = Matches(topics_50_EN['en'], topics_50_EN['hi'])
en_gu_matches = Matches(topics_50_EN['en'], topics_50_EN['gu'])
en_kn_matches = Matches(topics_50_EN['en'], topics_50_EN['kn'])
en_ml_matches = Matches(topics_50_EN['en'], topics_50_EN['ml'])
en_mr_matches = Matches(topics_50_EN['en'], topics_50_EN['mr'])
en_or_matches = Matches(topics_50_EN['en'], topics_50_EN['or'])
en_pa_matches = Matches(topics_50_EN['en'], topics_50_EN['pa'])
en_ta_matches = Matches(topics_50_EN['en'], topics_50_EN['ta'])
en_te_matches = Matches(topics_50_EN['en'], topics_50_EN['te'])


matches_50_EN = {'as': en_as_matches.score(), 'bn': en_bn_matches.score(), 
             'en': en_hi_matches.score(), 'gu': en_gu_matches.score(),
             'kn': en_kn_matches.score(),
             'ml': en_ml_matches.score(), 'mr': en_mr_matches.score(),
             'or': en_or_matches.score(), 'pa': en_pa_matches.score(),
             'ta': en_ta_matches.score(), 'te': en_te_matches.score()}
matches_50_EN

In [ ]:
# Average Matches for 50 topics
average_matches_50_EN = sum(matches_50_EN.values())/len(matches_50_EN.keys())

2. **Distributional Similarity**
> Compute the KL divergence between the predicted topic distribution on the test document and the same test document in English. Lower scores are better, indicating that the distributions do not differ by much.

## Distributional Similarity (KL Divergence)

### Hindi

In [ ]:
# HINDI : KL Divergence for 25 topics
hi_as_kl = KLDivergence(topics_25_HI['hi'], topics_25_HI['as'])
hi_bn_kl = KLDivergence(topics_25_HI['hi'], topics_25_HI['bn'])
hi_en_kl = KLDivergence(topics_25_HI['hi'], topics_25_HI['en'])
hi_gu_kl = KLDivergence(topics_25_HI['hi'], topics_25_HI['gu'])
hi_kn_kl = KLDivergence(topics_25_HI['hi'], topics_25_HI['kn'])
hi_ml_kl = KLDivergence(topics_25_HI['hi'], topics_25_HI['ml'])
hi_mr_kl = KLDivergence(topics_25_HI['hi'], topics_25_HI['mr'])
hi_or_kl = KLDivergence(topics_25_HI['hi'], topics_25_HI['or'])
hi_pa_kl = KLDivergence(topics_25_HI['hi'], topics_25_HI['pa'])
hi_ta_kl = KLDivergence(topics_25_HI['hi'], topics_25_HI['ta'])
hi_te_kl = KLDivergence(topics_25_HI['hi'], topics_25_HI['te'])

kl_divergence_25_HI = {'as': hi_as_kl.score(), 'bn': hi_bn_kl.score(), 
             'en': hi_en_kl.score(), 'gu': hi_gu_kl.score(),
             'kn': hi_kn_kl.score(),
             'ml': hi_ml_kl.score(), 'mr': hi_mr_kl.score(),
             'or': hi_or_kl.score(), 'pa': hi_pa_kl.score(),
             'ta': hi_ta_kl.score(), 'te': hi_te_kl.score()}

kl_divergence_25_HI

In [ ]:
# Average KLD for 25 topics
average_kl_divergence_25_HI = sum(kl_divergence_25_HI.values())/len(kl_divergence_25_HI.keys())

In [ ]:
# HINDI : KL Divergence for 50 topics
hi_as_kl = KLDivergence(topics_50_HI['hi'], topics_50_HI['as'])
hi_bn_kl = KLDivergence(topics_50_HI['hi'], topics_50_HI['bn'])
hi_en_kl = KLDivergence(topics_50_HI['hi'], topics_50_HI['en'])
hi_gu_kl = KLDivergence(topics_50_HI['hi'], topics_50_HI['gu'])
hi_kn_kl = KLDivergence(topics_50_HI['hi'], topics_50_HI['kn'])
hi_ml_kl = KLDivergence(topics_50_HI['hi'], topics_50_HI['ml'])
hi_mr_kl = KLDivergence(topics_50_HI['hi'], topics_50_HI['mr'])
hi_or_kl = KLDivergence(topics_50_HI['hi'], topics_50_HI['or'])
hi_pa_kl = KLDivergence(topics_50_HI['hi'], topics_50_HI['pa'])
hi_ta_kl = KLDivergence(topics_50_HI['hi'], topics_50_HI['ta'])
hi_te_kl = KLDivergence(topics_50_HI['hi'], topics_50_HI['te'])

kl_divergence_50_HI = {'as': hi_as_kl.score(), 'bn': hi_bn_kl.score(), 
             'en': hi_en_kl.score(), 'gu': hi_gu_kl.score(),
             'kn': hi_kn_kl.score(),
             'ml': hi_ml_kl.score(), 'mr': hi_mr_kl.score(),
             'or': hi_or_kl.score(), 'pa': hi_pa_kl.score(),
             'ta': hi_ta_kl.score(), 'te': hi_te_kl.score()}

kl_divergence_50_HI

In [ ]:
# Average KLD for 50 topics
average_kl_divergence_50_HI = sum(kl_divergence_50_HI.values())/len(kl_divergence_50_HI.keys())

### English

In [ ]:
# ENGLISH : KL Divergence for 25 topics
en_as_kl = KLDivergence(topics_25_EN['en'], topics_25_EN['as'])
en_bn_kl = KLDivergence(topics_25_EN['en'], topics_25_EN['bn'])
en_hi_kl = KLDivergence(topics_25_EN['en'], topics_25_EN['hi'])
en_gu_kl = KLDivergence(topics_25_EN['en'], topics_25_EN['gu'])
en_kn_kl = KLDivergence(topics_25_EN['en'], topics_25_EN['kn'])
en_ml_kl = KLDivergence(topics_25_EN['en'], topics_25_EN['ml'])
en_mr_kl = KLDivergence(topics_25_EN['en'], topics_25_EN['mr'])
en_or_kl = KLDivergence(topics_25_EN['en'], topics_25_EN['or'])
en_pa_kl = KLDivergence(topics_25_EN['en'], topics_25_EN['pa'])
en_ta_kl = KLDivergence(topics_25_EN['en'], topics_25_EN['ta'])
en_te_kl = KLDivergence(topics_25_EN['en'], topics_25_EN['te'])

kl_divergence_25_EN = {'as': en_as_kl.score(), 'bn': en_bn_kl.score(), 
             'hi': en_hi_kl.score(), 'gu': en_gu_kl.score(),
             'kn': en_kn_kl.score(),
             'ml': en_ml_kl.score(), 'mr': en_mr_kl.score(),
             'or': en_or_kl.score(), 'pa': en_pa_kl.score(),
             'ta': en_ta_kl.score(), 'te': en_te_kl.score()}

kl_divergence_25_EN

In [ ]:
# Average KLD for 50 topics
average_kl_divergence_25_EN = sum(kl_divergence_25_EN.values())/len(kl_divergence_25_EN.keys())

In [ ]:
# ENGLISH : KL Divergence for 50 topics
en_as_kl = KLDivergence(topics_50_EN['en'], topics_50_EN['as'])
en_bn_kl = KLDivergence(topics_50_EN['en'], topics_50_EN['bn'])
en_hi_kl = KLDivergence(topics_50_EN['en'], topics_50_EN['hi'])
en_gu_kl = KLDivergence(topics_50_EN['en'], topics_50_EN['gu'])
en_kn_kl = KLDivergence(topics_50_EN['en'], topics_50_EN['kn'])
en_ml_kl = KLDivergence(topics_50_EN['en'], topics_50_EN['ml'])
en_mr_kl = KLDivergence(topics_50_EN['en'], topics_50_EN['mr'])
en_or_kl = KLDivergence(topics_50_EN['en'], topics_50_EN['or'])
en_pa_kl = KLDivergence(topics_50_EN['en'], topics_50_EN['pa'])
en_ta_kl = KLDivergence(topics_50_EN['en'], topics_50_EN['ta'])
en_te_kl = KLDivergence(topics_50_EN['en'], topics_50_EN['te'])

kl_divergence_50_EN = {'as': en_as_kl.score(), 'bn': en_bn_kl.score(), 
             'hi': en_hi_kl.score(), 'gu': en_gu_kl.score(),
             'kn': en_kn_kl.score(),
             'ml': en_ml_kl.score(), 'mr': en_mr_kl.score(),
             'or': en_or_kl.score(), 'pa': en_pa_kl.score(),
             'ta': en_ta_kl.score(), 'te': en_te_kl.score()}

kl_divergence_50_EN

In [ ]:
# Average KLD for 50 topics
average_kl_divergence_50_EN = sum(kl_divergence_50_EN.values())/len(kl_divergence_50_EN.keys())

3. **Centroid Embeddings**
> To also account for similar but not exactly equal topic predictions, we compute the centroid embeddings of the 5 words describing the predicted topic for both English and non-English documents. Then we compute the cosine similarity between those two centroids (CD).

## Centroid Embeddings Distance (CD)

In [ ]:
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models import KeyedVectors
import gensim.downloader as api
from scipy.spatial.distance import cosine
import abc
import numpy as np

class CD(CentroidDistance):
    """Override author's function to upgrade compatibility with Gensim 4.0.0.
    See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4."""

    def get_centroid(self, word_list):
        vector_list = []
        for word in word_list:
            if word in self.wv:   # changed from self.wv.vocab to self.wv as in Gensim 4.0.0
                vector_list.append(self.wv.get_vector(word))
        vec = sum(vector_list)
        return vec / np.linalg.norm(vec)

### Hindi

In [ ]:
# HINDI : Centroid Embeddings for 25 topics
cd_25_HI = {}

for key in topics_25_HI.keys():
  if key == 'hi':
    continue
  topic = topics_25_HI[key]
  cd = CD(doc_distribution_original_language = topics_25_HI['hi'], 
          doc_distribution_unseen_language = topic, 
          topics = z_ctm_25_HI.get_topic_lists(25),
          topk = 5)
  
  cd_25_HI[key] = cd.score()

cd_25_HI

In [ ]:
# Average KLD for 25 topics
average_cd_25_HI = sum(cd_25_HI.values())/len(cd_25_HI.keys())

In [ ]:
# HINDI : Centroid Embeddings for 50 topics
cd_50_HI = {}

for key in topics_50_HI.keys():
  if key == 'hi':
    continue
  topic = topics_50_HI[key]
  cd = CD(doc_distribution_original_language = topics_50_HI['hi'], 
          doc_distribution_unseen_language = topic, 
          topics = z_ctm_50_HI.get_topic_lists(50),
          topk = 5)
  
  cd_50_HI[key] = cd.score()
  cd = None

cd_50_HI

### English

In [ ]:
# Average KLD for 25 topics
average_cd_25_HI = sum(cd_25_HI.values())/len(cd_25_HI.keys())

In [ ]:
# ENGLISH : Centroid Embeddings for 25 topics
cd_25_EN = {}

for key in topics_25_EN.keys():
  if key == 'en':
    continue
  topic = topics_25_EN[key]
  cd = CD(doc_distribution_original_language = topics_25_EN['en'], 
          doc_distribution_unseen_language = topic, 
          topics = z_ctm_25_EN.get_topic_lists(25),
          topk = 5)
  
  cd_25_EN[key] = cd.score()

cd_25_EN

In [ ]:
# ENGLISH : Centroid Embeddings for 50 topics
cd_50_EN = {}

for key in topics_50_EN.keys():
  if key == 'hi':
    continue
  topic = topics_50_EN[key]
  cd = CD(doc_distribution_original_language = topics_50_EN['en'], 
          doc_distribution_unseen_language = topic, 
          topics = z_ctm_50_EN.get_topic_lists(50),
          topk = 5)
  
  cd_50_EN[key] = cd.score()
  cd = None

cd_50_EN

In [ ]:
# Store Metrics
metrics = {
          "Hindi" : [{
                    "Mat25": matches_25_HI,
                    "KL25": kl_divergence_25_HI, 
                    "CD25": cd_25_HI, 
                    "Mat50": matches_50_HI, 
                    "KL50": kl_divergence_50_HI,
                    "CD50": cd_50_HI
                    }],

          "English": [{
                    "Mat25": matches_25_EN,
                    "KL25": kl_divergence_25_EN, 
                    "CD25": cd_25_EN, 
                    "Mat50": matches_50_EN, 
                    "KL50": kl_divergence_50_EN,
                    "CD50": cd_50_EN
                    }]
          }
with open("metrics_samescript.txt", 'wb') as F:
  pickle.dump(metrics, F)

metrics

# Evaluation Results

In [ ]:
# Show results
metrics = {"Hindi" : 
           
           {"Mat25": matches_25_HI,
           "KL25": kl_divergence_25_HI, 
           "CD25": cd_25_HI, 
           "Mat50": matches_50_HI, 
           "KL50": kl_divergence_50_HI,
           "CD50": cd_50_HI},
           
           "English" : 
           {"Mat25": matches_25_EN,
           "KL25": kl_divergence_25_EN, 
           "CD25": cd_25_EN, 
           "Mat50": matches_50_EN, 
           "KL50": kl_divergence_50_EN,
           "CD50": cd_50_EN}
           }

metrics = pd.DataFrame.from_dict(metrics, orient='columns') 
print("Match, KL, and Centroid Similarity for 25 and 50 topics on various languages on PMIndia Corpus")
metrics